In [1]:
import pandas as pd
danijels = pd.read_csv("011_segments.csv")
danijels.head(2)

,file,start,end,predicted
0,audio/58xZSVbpgkk.wav,11.5,40.1,nekoliko puta smo ponavljali da nio od uvođenj...
1,audio/58xZSVbpgkk.wav,40.1,62.5,nije problem u narodni poslanicima mi jeste pr...


In [2]:
danijels.file.unique().shape

(143,)

In [21]:
corpus = pd.read_csv("006_crawling_juznevesti.csv")
corpus["text"] = corpus.transcript
corpus.head(2)

,url,yt_hash,transcript,exists,path,text
0,https://www.juznevesti.com/15-minuta/Boban-Sto...,00DtyZ2sCUM,"Gost: Boban Stojanović, redovni profesor Ekono...",True,audio/00DtyZ2sCUM.wav,"Gost: Boban Stojanović, redovni profesor Ekono..."
1,https://www.juznevesti.com/15-minuta/Slavoljub...,58xZSVbpgkk,"Gost: Slavoljub Vlajković, narodni poslanik SP...",True,audio/58xZSVbpgkk.wav,"Gost: Slavoljub Vlajković, narodni poslanik SP..."


In [22]:
!wget https://github.com/danijel3/TextNormalize/raw/master/hrvatski.py
!wget https://github.com/danijel3/TextNormalize/raw/master/strings_hr.py

--2022-06-14 08:34:04--  https://github.com/danijel3/TextNormalize/raw/master/hrvatski.py
Resolving www-proxy.ijs.si (www-proxy.ijs.si)... 2001:1470:ff80::3128:1, 193.2.4.4
Connecting to www-proxy.ijs.si (www-proxy.ijs.si)|2001:1470:ff80::3128:1|:8080... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/danijel3/TextNormalize/master/hrvatski.py [following]
--2022-06-14 08:34:04--  https://raw.githubusercontent.com/danijel3/TextNormalize/master/hrvatski.py
Connecting to www-proxy.ijs.si (www-proxy.ijs.si)|2001:1470:ff80::3128:1|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 5761 (5.6K) [text/plain]
Saving to: ‘hrvatski.py.1’

hrvatski.py.1       100%[===================>]   5.63K  --.-KB/s    in 0s      

2022-06-14 08:34:05 (72.1 MB/s) - ‘hrvatski.py.1’ saved [5761/5761]

--2022-06-14 08:34:05--  https://github.com/danijel3/TextNormalize/raw/master/strings_hr.py
Resolving www-proxy.ijs.si (www-proxy.ij

In [23]:
from hrvatski import normalize
from dataclasses import dataclass, field
from typing import Dict, Set, List, Tuple


@dataclass
class Dictionary:
    word2id: Dict[str, int] = field(default_factory=lambda: {})
    id2word: Dict[int, str] = field(default_factory=lambda: {})
    oov_token: str = "<unk>"

    def put(self, words: Set):
        for id, word in enumerate(sorted(list(words))):
            self.word2id[word] = id
            self.id2word[id] = word

    def get_id(self, word: str, warn_oov: bool = False) -> int:
        if word not in self.word2id:
            if warn_oov:
                print(f'WARN: missing word "{word}"')
            return -1
        return self.word2id[word]

    def get_word(self, id: int) -> str:
        if id == -1:
            return self.oov_token
        return self.id2word[id]

    def get_text(self, ids: List[int]) -> str:
        return " ".join([self.get_word(x) for x in ids])

    def get_ids(self, text: str, warn_oov: bool = False) -> List[int]:
        return [self.get_id(x, warn_oov) for x in text.strip().split()]


def load_ref(file):
    lines = []
    text = []
    words = set()
    vocab = Dictionary()
    with open(file) as f:
        for l in f:
            tok = normalize(l.strip()).strip().split()
            lines.append(tok)
            words.update(tok)
        vocab.put(words)
        for l in lines:
            for w in l:
                text.append(vocab.get_id(w))
    return text, vocab


In [24]:
file = danijels.file[0]
c = corpus.path == file
transcripts = " ".join(corpus.transcript[c].values)
from pathlib import Path
Path("012_test_transcripts.txt").write_text(transcripts)

13714

In [25]:
text,vocab=load_ref('012_test_transcripts.txt')

In [26]:
print(text[:10])
print(vocab.get_text(text[:10]))

[115, 634, 776, 327, 499, 657, 174, 382, 782, 531]
gost slavoljub vlajković narodni poslanik sps iz nišarazgovor vodio predrag


In [38]:
danijels

,file,start,end,predicted
0,audio/58xZSVbpgkk.wav,11.5,40.1,nekoliko puta smo ponavljali da nio od uvođenj...
1,audio/58xZSVbpgkk.wav,40.1,62.5,nije problem u narodni poslanicima mi jeste pr...
2,audio/58xZSVbpgkk.wav,62.5,82.1,znači grad uopštenije organizvan i mi faktički...
3,audio/58xZSVbpgkk.wav,82.1,105.7,pa iima ali ja kažem da bi trebao grada kažem ...
4,audio/58xZSVbpgkk.wav,105.7,134.9,ja sam skoro slušao neke podatkee prvi kliničk...
...,...,...,...,...
5536,audio/IUIKV9XlWEM.wav,1526.4,1552.8,i zapravo se ispostavilo i čovjek je liepo ogl...
5537,audio/IUIKV9XlWEM.wav,1552.8,1578.7,odnosima bivše jugoslavije ovako ili onako sam...
5538,audio/IUIKV9XlWEM.wav,1578.7,1608.1,bogati hoće ili silni hoće da zavede neku siro...
5539,audio/IUIKV9XlWEM.wav,1608.1,1636.8,da budete prije svega naši dragi gosti a posto...


In [41]:
import json
danijels = danijels.rename(columns={"predicted": "text"})
output = json.loads(danijels.loc[danijels.file==file,:].to_json(orient="records"))

In [43]:
from sys import int_info
from difflib import SequenceMatcher


def findall(id, sequence):
    ret = []
    off = 0
    N = len(sequence)
    while off < N:
        try:
            pos = sequence.index(id, off)
            ret.append(pos)
            off = pos + 1
        except ValueError:
            break
    return ret


def close_match(ids: List[int], text: List[int]) -> Tuple[int, int]:
    N = len(ids)
    p1 = findall(ids[0], text)
    poff = 1
    while len(p1) == 0 and poff < N:
        p2 = findall(ids[poff], text)
        p1 = [x - poff for x in p2]
        poff += 1
    max_r = 0
    pf = []
    for p in p1:
        sm = SequenceMatcher(a=ids, b=text[p : p + N], autojunk=False)
        r = sm.ratio()
        if r > max_r:
            max_r = r
            pf = [p]
        elif r == max_r:
            pf.append(p)

    if len(pf) == 0:
        print("ERROR: no candidates found!")
        return None
    elif len(pf) > 1:
        print("WARNING: multiple candidates found!")

    pf = pf[0]

    mb = SequenceMatcher(
        a=ids, b=text[pf : pf + N + 10], autojunk=False
    ).get_matching_blocks()
    m = mb[-2]
    M = m.b + m.size

    return pf, pf + M


In [44]:
for seg in output:
    ids = vocab.get_ids(seg["text"])
    ps, pe = close_match(ids, text)
    print("ASR: " + seg["text"])
    print("REF: " + vocab.get_text(text[ps:pe]))


ASR: nekoliko puta smo ponavljali da nio od uvođenja više stranačja nije nikad imao veći broj narodnih poslanika u skupštini srbiji razgovaramo sa slavoljevom vlajkovićem poslanikom socijalistički partije srbije gospodine vlajkoviću kakav je vaš utjescak da li je niž iskoristio to što ili koristi to što ima 12 poslanika nikad više pa niti iskoristio niti će izgledat iskoristit priliku štosida je to pa problem jeu nama mislim da je problem u gradu
REF: nekoliko puta smo ponavljali da niš od uvođenja višestranačja nijenikada imao veći broj poslanika u skupštini srbije razgovaramo sa slavoljubom vlajkovićem narodnim poslanikom socijalističke partije srbije gospodine vlajkoviću kakav je vaš utisak da li je niš iskoristio to što ili koristi to što ima dvanaest poslanika nikad više niti je iskoristio niti će izgleda iskoristi priliku mislim da je to jv šta je problem pa problem je u nama mislim da je problem u gradu
ASR: nije problem u narodni poslanicima mi jeste predstavno u svoje stranke 